In [1]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
import glob

In [3]:
import numpy as np

In [4]:
from scipy.interpolate import interp1d
from scipy.interpolate import spline

In [5]:
from tqdm import tqdm

In [6]:
DQN_Rewards_Path = "/home/tabz/Dropbox/RL/Log_Files/March 2017/Maze_5_Batch_Sizes_Logs/*DQN*/logs/Episode_Rewards.txt"
DQN_Lengths_Path = "/home/tabz/Dropbox/RL/Log_Files/March 2017/Maze_5_Batch_Sizes_Logs/*DQN*/logs/Episode_Lengths.txt"
DQN_Rewards = []
DQN_Lengths = []
DQN_Lengths_C = []

In [7]:
for filename in glob.glob(DQN_Rewards_Path):
    run_logs = [0]
#     print(filename)
    with open(filename, "r") as f:
        for line in f:
            try:
                ep_r = float(line)
            except:
                continue
            run_logs.append(ep_r)
    DQN_Rewards.append(run_logs)

In [8]:
for filename in glob.glob(DQN_Lengths_Path):
    run_logs = [0]
#     print(filename)
    with open(filename, "r") as f:
        for line in f:
            try:
                ep_r = float(line)
            except:
                continue
            run_logs.append(ep_r)
    DQN_Lengths.append(run_logs)
    run_logs_c = np.cumsum(run_logs).astype(np.int32)
    DQN_Lengths_C.append(run_logs_c)

for lens, runs in zip(DQN_Lengths_C, DQN_Rewards):
    plt.plot(lens, runs)
plt.show()

p = figure(width=500, height=500, y_range=(-1,3))
for lens, runs in zip(DQN_Lengths_C, DQN_Rewards):
    p.line(lens, runs)
show(p)

def plot_averages(names, t_maxs, colors, legends=None, directory="Logs/", step=10):
    plt.figure(figsize=(10,10))
    plt.ylim(-1, 3)
    for name, color, t_max in tqdm(zip(names, colors, t_maxs)):
        # TODO: Remove DQN from names
        DQN_Rewards_Path = directory + "*" + name + "*/logs/Episode_Rewards.txt"
        DQN_Lengths_Path = directory + "*" + name + "*/logs/Episode_Lengths.txt"
        DQN_Rewards = []
        DQN_Lengths = []
        DQN_Lengths_C = []
        for filename in glob.glob(DQN_Rewards_Path):
            run_logs = [0]
            with open(filename, "r") as f:
                for line in f:
                    try:
                        ep_r = float(line)
                    except:
                        continue
                    run_logs.append(ep_r)
            DQN_Rewards.append(run_logs)
        for filename in glob.glob(DQN_Lengths_Path):
            run_logs = [0]
        #     print(filename)
            with open(filename, "r") as f:
                for line in f:
                    try:
                        ep_r = float(line)
                    except:
                        continue
                    run_logs.append(ep_r)
            DQN_Lengths.append(run_logs)
            run_logs_c = np.cumsum(run_logs).astype(np.int32)
            DQN_Lengths_C.append(run_logs_c)
        DQN_Smoothed = []
        for times, rewards in zip(DQN_Lengths_C, DQN_Rewards):
#             linear = interp1d(times, rewards, kind="linear")
#             linear_rewards = linear([i for i in range(500000)])
#             DQN_Smoothed.append(linear_rewards)
            spline_rewards = spline(times, rewards, [i for i in range(0, t_max, step)])
            DQN_Smoothed.append(spline_rewards)
        DQN_Means = np.mean(DQN_Smoothed, axis=0)
        DQN_Stds = np.std(DQN_Smoothed, axis=0)
        plt.fill_between([i for i in range(0, t_max, step)], DQN_Means - DQN_Stds, DQN_Means + DQN_Stds, color=color, alpha=0.2, edgecolor="white")

        plt.plot([i for i in range(0, t_max, step)], DQN_Means, color=color)

    plt.title("Episode_Reward")
    plt.xlabel("T")
    plt.ylabel("Reward")
    if legends is not None:
        plt.legend(legends, bbox_to_anchor=(1.05, 1), loc=2)
    plt.show()

In [9]:
from bokeh.models import Legend
from bokeh.models.formatters import BasicTickFormatter
from bokeh.models import HoverTool, TapTool

In [10]:
def plot_averages_b(names, t_maxs, colors, legends, directories="Logs/", step=10, title=""):
#     plt.figure(figsize=(10,10))
    p = figure(width=800, height=800, y_range=(-1.2, 3.2), title=title)
    p.toolbar_location = "above"
    p.yaxis.axis_label = "Episode Reward"
    p.xaxis.axis_label = "T"
    p.xaxis.formatter = BasicTickFormatter(use_scientific=False)
#     p = figure(width=800, height=500)
    lines = []
    alpha_lines = []
    if isinstance(directories, str):
        dd = directories
        directories = [dd for _ in names]
    for name, color, t_max, legend, directory in zip(names, colors, t_maxs, legends, directories):
#         print(directory)
        DQN_Rewards_Path = directory + "*" + name + "*/logs/Episode_Rewards.txt"
        DQN_Lengths_Path = directory + "*" + name + "*/logs/Episode_Lengths.txt"
        DQN_Rewards = []
        DQN_Lengths = []
        DQN_Lengths_C = []
        for filename in glob.glob(DQN_Rewards_Path):
            run_logs = [0]
            with open(filename, "r") as f:
#                 print(f)
                for line in f:
                    try:
                        ep_r = float(line)
                    except:
                        print("Nothing here for:", filename)
                        continue
                    run_logs.append(ep_r)
            DQN_Rewards.append(run_logs)
        for filename in glob.glob(DQN_Lengths_Path):
            run_logs = [0]
        #     print(filename)
            with open(filename, "r") as f:
                for line in f:
                    try:
                        ep_r = float(line)
                    except:
                        continue
                    run_logs.append(ep_r)
            DQN_Lengths.append(run_logs)
            run_logs_c = np.cumsum(run_logs).astype(np.int32)
            DQN_Lengths_C.append(run_logs_c)
        DQN_Smoothed = []
        for times, rewards in zip(DQN_Lengths_C, DQN_Rewards):
#             linear = interp1d(times, rewards, kind="linear")
#             linear_rewards = linear([i for i in range(500000)])
#             DQN_Smoothed.append(linear_rewards)
            spline_rewards = spline(times, rewards, [i for i in range(0, t_max, step)], order=1)
            DQN_Smoothed.append(spline_rewards)
        DQN_Means = np.mean(DQN_Smoothed, axis=0)
        DQN_Stds = np.std(DQN_Smoothed, axis=0)
        
#         plt.fill_between([i for i in range(0, t_max, step)], DQN_Means - DQN_Stds, DQN_Means + DQN_Stds, color=color, alpha=0.2, edgecolor="white")
        # Error bars
        xs = [i for i in range(0, t_max, step)]
        xs = xs + list(reversed(xs))
        ys = np.concatenate([DQN_Means - DQN_Stds, np.flip(DQN_Means + DQN_Stds, axis=0)])
        lls = p.patch(xs, ys, color=color, alpha=0.1)

        
#         for x, y, yerr in zip(range(0, t_max, step), DQN_Means, DQN_Stds):
#             lls = p.line([x, x], [y - yerr, y + yerr], color=color, alpha=0.1, hover_alpha=0.8)
#             alpha_lines.append(lls)
            

#         plt.plot([i for i in range(0, t_max, step)], DQN_Means, color=color)
        l = p.line([i for i in range(0, t_max, step)], DQN_Means, color=color, line_width=2)
        lines.append(l)
#         alpha_lines.append(lls)

#     plt.title("Episode_Reward")
#     plt.xlabel("T")
#     plt.ylabel("Reward")
#     if legends is not None:
#         plt.legend(legends)
#     plt.show()
#     p.line([0,1], [-1,3])
    new_leg = Legend(items=[(name, [line]) for name, line in zip(legends, lines)], location=(0,-30))
    p.add_layout(new_leg, "right")
#     p.add_tools(HoverTool(renderers=alpha_lines))
#     p.add_tools(TapTool(renderers=alpha_lines))
    show(p)

In [41]:
def plot_states(names, t_maxs, colors, legends, directories="Logs/", step=10, title=""):
#     plt.figure(figsize=(10,10))
    p = figure(width=1200, height=800, title=title)
    p.toolbar_location = "above"
    p.yaxis.axis_label = "States visited"
    p.xaxis.axis_label = "T"
    p.xaxis.formatter = BasicTickFormatter(use_scientific=False)
#     p = figure(width=800, height=500)
    lines = []
    alpha_lines = []
    if isinstance(directories, str):
        dd = directories
        directories = [dd for _ in names]
    for name, color, t_max, legend, directory in zip(names, colors, t_maxs, legends, directories):
#         print(directory)
        States_Path = directory + "*" + name + "*/logs/Player_Positions.txt"
        States = []
        for filename in glob.glob(States_Path):
            run_logs = []
            with open(filename, "r") as f:
#                 print(f)
                for line in f:
                    try:
                        ep_r = str(line)
#                         print(ep_r)
                    except:
                        print("Nothing here for:", filename)
                        continue
                    run_logs.append(ep_r)
            States.append(run_logs)
        States_Visited = []
        for run in States:
            Visited = []
            States_Sets = set()
#             print(len(run))
            for ii, s in enumerate(run):
                if ii >= t_max - 1:
                    break
#                 print(s)
                States_Sets.add(s)
                if ii % step == 0:
                    Visited.append(len(States_Sets))
#             print(len(States_Sets))
#             print(len(Visited))
            States_Visited.append(Visited)
#         print(np.array(States_Visited).shape)
        Means = np.mean(States_Visited, axis=0)
        Stds = np.std(States_Visited, axis=0)
#         print(Means.shape)
        
#         plt.fill_between([i for i in range(0, t_max, step)], DQN_Means - DQN_Stds, DQN_Means + DQN_Stds, color=color, alpha=0.2, edgecolor="white")
        # Error bars
        xs = [i for i in range(0, t_max, step)]
        xs = xs + list(reversed(xs))
        ys = np.concatenate([Means - Stds, np.flip(Means + Stds, axis=0)])
        lls = p.patch(xs, ys, color=color, alpha=0.1)

        
#         for x, y, yerr in zip(range(0, t_max, step), DQN_Means, DQN_Stds):
#             lls = p.line([x, x], [y - yerr, y + yerr], color=color, alpha=0.1, hover_alpha=0.8)
#             alpha_lines.append(lls)
            

#         plt.plot([i for i in range(0, t_max, step)], DQN_Means, color=color)
        l = p.line([i for i in range(0, t_max, step)], Means, color=color, line_width=2)
        lines.append(l)
#         alpha_lines.append(lls)

#     plt.title("Episode_Reward")
#     plt.xlabel("T")
#     plt.ylabel("Reward")
#     if legends is not None:
#         plt.legend(legends)
#     plt.show()
#     p.line([0,1], [-1,3])
    new_leg = Legend(items=[(name, [line]) for name, line in zip(legends, lines)], location=(0,-30))
    p.add_layout(new_leg, "right")
#     p.add_tools(HoverTool(renderers=alpha_lines))
#     p.add_tools(TapTool(renderers=alpha_lines))
    show(p)

In [127]:
names = []
legends = []
t_maxs = []
directories = []

            
# Frontier stuff
for step in [10]:
    # DQN
    names += ["_{}_Step_*DQN_".format(step)]
    legends += ["{}_Step_DQN".format(step)]
    t_maxs += [300001]
#     continue
    # PseudoCount
    names += ["_{}_Step_*k_Count_".format(step)]
    legends += ["{}_Step_Count".format(step)]
    t_maxs += [300001]
#     continue
    # EpsilonScaling
    names += ["_{}_Step_*_CountEps_".format(step)]
    legends += ["{}_Step_Epsilon_Scaling".format(step)]
    t_maxs += [300001]
            
    
direc = "/home/tabz/tmp/logfiles/MedMaze10/MedMaze10"
# directories =  ["{}/{}".format(dropbox_direc, "Maze_5_Logs/")] + 3 * ["{}/{}".format(dropbox_direc, "Maze_5_Batch_Sizes_Logs/")] 
directories += [direc + "/EpsilonLogs/"] * 3
    
print(names)
print(t_maxs)
# colours = ["red", "#3366ff", "#66ff33", "yellow"]
# colours = rainbow(np.linspace(0, 1, len(names)))

['_10_Step_*DQN_', '_10_Step_*k_Count_', '_10_Step_*_CountEps_']
[300001, 300001, 300001]


In [128]:
# names = []
# legends = []
# t_maxs = []

            
# Frontier bandit stuff

# ForceAction
names += ["ForceAction"]
legends += ["1_Step_ForceAction"]
t_maxs += [300001]

# Optimistic
names += ["OptimisticAction"]
legends += ["1_Step_OptimisticAction"]
t_maxs += [300001]

# State action pseudo counts
names += ["StateAction"]
legends += ["1_Step_Action_Count"]
t_maxs += [300001]
            
    
direc = "/home/tabz/tmp/logfiles/MedMaze10/MedMaze10"
# directories =  ["{}/{}".format(dropbox_direc, "Maze_5_Logs/")] + 3 * ["{}/{}".format(dropbox_direc, "Maze_5_Batch_Sizes_Logs/")] 
directories += [direc + "/BanditLogs/"] * 3
    
print(names)
print(t_maxs)
print(directories)
# colours = ["red", "#3366ff", "#66ff33", "yellow"]
# colours = rainbow(np.linspace(0, 1, len(names)))

['_10_Step_*DQN_', '_10_Step_*k_Count_', '_10_Step_*_CountEps_', 'ForceAction', 'OptimisticAction', 'StateAction']
[300001, 300001, 300001, 300001, 300001, 300001]
['/home/tabz/tmp/logfiles/MedMaze10/MedMaze10/EpsilonLogs/', '/home/tabz/tmp/logfiles/MedMaze10/MedMaze10/EpsilonLogs/', '/home/tabz/tmp/logfiles/MedMaze10/MedMaze10/EpsilonLogs/', '/home/tabz/tmp/logfiles/MedMaze10/MedMaze10/BanditLogs/', '/home/tabz/tmp/logfiles/MedMaze10/MedMaze10/BanditLogs/', '/home/tabz/tmp/logfiles/MedMaze10/MedMaze10/BanditLogs/']


In [129]:
import itertools

In [130]:
from bokeh.palettes import magma, inferno, Purples, Oranges, Greens, Reds, Set1, Set3, Paired, Dark2

In [131]:
plot_states(names, t_maxs, Set1[len(names)], legends, directories, step=1000, title="Med Maze 10")

/home/tabz/.local/lib/python3.6/site-packages/bokeh/models/sources.py:81: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/home/tabz/.local/lib/python3.6/site-packages/bokeh/models/sources.py:81: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/home/tabz/.local/lib/python3.6/site-packages/bokeh/models/sources.py:81: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/home/tabz/.local/lib/python3.6/site-packages/bokeh/models/sources.py:81: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/home/tabz/.loca

In [ ]:
DQN_Smoothed = []
for times, rewards in zip(DQN_Lengths_C, DQN_Rewards):
    linear = interp1d(times, rewards, kind="linear")
    linear_rewards = linear([i for i in range(500000)])
    DQN_Smoothed.append(linear_rewards)

In [ ]:
DQN_Means = np.mean(DQN_Smoothed, axis=0)

In [ ]:
DQN_Stds = np.std(DQN_Smoothed, axis=0)

In [ ]:
plt.fill_between([i for i in range(500000)], DQN_Means - DQN_Stds, DQN_Means + DQN_Stds, color="red", alpha=0.3, edgecolor="white")

plt.plot([i for i in range(500000)], DQN_Means, color="red")

In [ ]:
len(DQN_Repeat)

In [ ]:
plt.plot(DQN)

In [ ]:
directories = ["Logs/"]

In [ ]:
for i in range(0, 10, 2):
    print(i)